# Classification 

In [1]:
# ] add RDatasets

In [2]:
# ] add MLBase

In [3]:
using RDatasets
using MLBase

In [4]:
iris = dataset("datasets", "iris")

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
8,5.0,3.4,1.5,0.2,setosa
9,4.4,2.9,1.4,0.2,setosa


In [5]:
X = Matrix(iris[:,1:4])
irislabels = iris[:,5]

150-element CategoricalArrays.CategoricalArray{String,1,UInt8}:
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 ⋮
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"

In [6]:
irislabelsmap = labelmap(irislabels)

LabelMap (with 3 labels):
[1] setosa
[2] versicolor
[3] virginica


In [7]:
y = labelencode(irislabelsmap, irislabels)  # convertir las categorias a numeros enteros 

150-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In [8]:
using Random

In [9]:
function perclass_splits(y, p) # función para partición del dataframe
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids, p) 
        push!(keepids,rowids...)
    end
    return keepids
end

perclass_splits (generic function with 1 method)

In [10]:
train_ids = perclass_splits(y, 0.7) # particion del 70%

96-element Vector{Any}:
   2
   3
   4
   5
   6
   9
  13
  14
  15
  16
  18
  19
  20
   ⋮
 135
 136
 137
 139
 141
 142
 145
 146
 147
 148
 149
 150

In [11]:
test_ids = setdiff(1:length(y), train_ids)

54-element Vector{Any}:
   1
   7
   8
  10
  11
  12
  17
  21
  28
  29
  32
  33
  37
   ⋮
 113
 116
 119
 122
 128
 131
 132
 134
 138
 140
 143
 144

In [12]:
@show size(train_ids)
@show size(test_ids);

size(train_ids) = (96,)
size(test_ids) = (54,)


In [13]:
function assign_class(predictedvalue)
    return argmin(abs.(predictedvalue .- [1, 2, 3]))      # devuelve el valor estiado mas cercano a la categoria numérica 
end

assign_class (generic function with 1 method)

In [14]:
# para un valor estimado de 3.1: 
argmin([2.1, 1.1, 0.9])   

3

In [15]:
assign_class(2.2)

2

## Usando Clasificadores 

## Decision Trees / Árboles de Decision

In [16]:
# ] add DecisionTree

In [17]:
using DecisionTree

In [18]:
model = DecisionTreeClassifier(max_depth=2)

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  nothing
root:                     nothing

In [19]:
DecisionTree.fit!(model, X[train_ids,:], y[train_ids])

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  [1, 2, 3]
root:                     Decision Tree
Leaves: 3
Depth:  2

In [20]:
q = X[test_ids,:];
predictions_DT = DecisionTree.predict(model, q)

54-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 2
 2
 3
 3
 3
 3
 3
 3
 3

In [21]:
function findaccuracy(predictedvals, groundtruthvals)
    sum(predictedvals .== groundtruthvals) / length(groundtruthvals)
end 

findaccuracy (generic function with 1 method)

In [22]:
findaccuracy(predictions_DT,y[test_ids])

0.9444444444444444

### Random Forests 

In [23]:
model = RandomForestClassifier(n_trees=20)
DecisionTree.fit!(model, X[train_ids,:], y[train_ids])

RandomForestClassifier
n_trees:             20
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             [1, 2, 3]
ensemble:            Ensemble of Decision Trees
Trees:      20
Avg Leaves: 5.9
Avg Depth:  4.15

In [24]:
q = X[test_ids,:];
predictions_RF = DecisionTree.predict(model, q)
findaccuracy(predictions_RF,y[test_ids])

0.9814814814814815

### Support Vector Machines 

In [25]:
] add LIBSVM 

    Updating registry at `C:\Users\pmore\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\pmore\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\pmore\.julia\environments\v1.8\Manifest.toml`


In [26]:
using LIBSVM

In [27]:
Xtrain = X[train_ids,:]
ytrain = y[train_ids]

96-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In [ ]:
model = svmtrain(Xtrain', ytrain)

In [ ]:
predictions_SVM, decision_values = svmpredict(model, X[test_ids,:]')
findaccuracy(predictions_SVM, y[test_ids])